# Attempt to fine-tune GPT3 to translate between languages
The first language will be our made up language (see notes on get_bible as to why we think GPT is cheating)
The thesis is that fine-tuning will cause GPT3 to learn the fake language as a new language by assigning the embeddings of the
new words essentially to the same dimensional space as the english.  Should be trivial to translate it back as it is a word 
for word translation.



In [5]:
from lib.config import get_config
import pandas as pd

bible = pd.read_csv('data/bible.csv')

# Assuming the gospels have a lot of overlap include the synoptic gospels plus John to train
# then predict Mark (which should be easy as Matthew and Luke may have copied from him)
train = bible[bible['book'].isin(['MAT','LUK','JHN'])]
validate = bible[bible['book'].isin(['MRK'])]

In [6]:
train

,0,book,chapter,verse,eng-web,eng-asv,eng-kjv2006,engBBE,birrig
23213,MAT 1:1,MAT,1,1,"The book of the genealogy of Jesus Christ, the...","The book of the generation of Jesus Christ, th...","The book of the generation of Jesus Christ, th...","The book of the generations of Jesus Christ, t...","Lxi fuuc us lxi girivelourz us Yizaz Kxvozl, l..."
23214,MAT 1:2,MAT,1,2,Abraham became the father of Isaac. Isaac beca...,Abraham begat Isaac; and Isaac begat Jacob; an...,Abraham begat Isaac; and Isaac begat Jacob; an...,The son of Abraham was Isaac; and the son of I...,Lxi zur us Efvexen hez Ozeek; erw lxi zur us O...
23215,MAT 1:3,MAT,1,3,Judah became the father of Perez and Zerah by ...,and Judah begat Perez and Zerah of Tamar; and ...,And Judas begat Phares and Zara of Thamar; and...,And the sons of Judah were Perez and Zerah by ...,Erw lxi zurz us Yawex hivi Qivid erw Divex fj ...
23216,MAT 1:4,MAT,1,4,Ram became the father of Amminadab. Amminadab ...,and Ram begat Amminadab; and Amminadab begat N...,And Aram begat Aminadab; and Aminadab begat Na...,And the son of Ram was Amminadab; and the son ...,Erw lxi zur us Ven hez Ennorewef; erw lxi zur ...
23217,MAT 1:5,MAT,1,5,Salmon became the father of Boaz by Rahab. Boa...,and Salmon begat Boaz of Rahab; and Boaz begat...,And Salmon begat Booz of Rachab; and Booz bega...,And the son of Salmon by Rahab was Boaz; and t...,Erw lxi zur us Zepnur fj Vexef hez Fued; erw l...
...,...,...,...,...,...,...,...,...,...
26987,JHN 21:21,JHN,21,21,"Peter, seeing him, said to Jesus, “Lord, what ...","Peter therefore seeing him saith to Jesus, Lor...","Peter seeing him saith to Jesus, Lord, and wha...","Seeing him, Peter said to Jesus, What about th...","Ziiorg xon, Qiliv zeow lu Yizaz, Hxel efual lx..."
26988,JHN 21:22,JHN,21,22,"Jesus said to him, “If I desire that he stay u...","Jesus saith unto him, If I will that he tarry ...","Jesus saith unto him, If I will that he tarry ...","Jesus said to him, If it is my desire for him ...","Yizaz zeow lu xon, Os ol oz nj wizovi suv xon ..."
26989,JHN 21:23,JHN,21,23,This saying therefore went out among the broth...,This saying therefore went forth among the bre...,Then went this saying abroad among the brethre...,So this saying went about among the brothers t...,Zu lxoz zejorg hirl efual enurg lxi fvulxivz l...
26990,JHN 21:24,JHN,21,24,This is the disciple who testifies about these...,This is the disciple that beareth witness of t...,This is the disciple which testifieth of these...,This is the disciple who gives witness about t...,Lxoz oz lxi wozkoqpi hxu gomiz holrizz efual l...


In [7]:
df2 = pd.DataFrame({'prompt': train['engBBE'], 'completion': train['birrig']})
df2['completion'] = df2['completion'].apply(lambda x: " " + x + '\n###')
df2['prompt'] = df2['prompt'].apply(lambda x: x + "\n\n###\n\n")
df2.to_json('data/train.jsonl', orient='records', lines=True)


In [12]:

# set environment variable in ipython notebook
import os
os.environ["OPENAI_API_KEY"] = get_config('openai')['api_key']




In [20]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py tools fine_tunes.prepare_data -f data/train.jsonl -q

Analyzing...

- Your file contains 3101 prompt-completion pairs
- There are 13 duplicated prompt-completion sets. These are rows: [403, 593, 787, 886, 1364, 1395, 1399, 1670, 1675, 1864, 2036, 2129, 2391]
- All prompts end with suffix `\n\n###\n\n`
- All completions end with suffix `\n###`

Based on the analysis we will perform the following actions:
- [Recommended] Remove 13 duplicate rows [Y/n]: Y


Your data will be written to a new JSONL file. Proceed [Y/n]: Y

Wrote modified file to `data/train_prepared.jsonl`
Feel free to take a look!

Now use that file when fine-tuning:
> openai api fine_tunes.create -t "data/train_prepared.jsonl"

After you’ve fine-tuned a model, remember that your prompt has to end with the indicator string `\n\n###\n\n` for the model to start generating completions, rather than continuing with the prompt. Make sure to include `stop=["\n###"]` so that the generated texts ends at the expected place.
Once your model starts training, it'll approximately take 2.04

In [21]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py api fine_tunes.create -t "data/train_prepared.jsonl"

Upload progress: 100%|███████████████████████| 871k/871k [00:00<00:00, 650Mit/s]
Uploaded file from data/train_prepared.jsonl: file-Kz9OeIePo1hXszuPaZ12yBeU
Created fine-tune: ft-MPF2xw4JKKURr7gU4EsxRERI
Streaming events until fine-tuning is complete...

(Ctrl-C will interrupt the stream, but not cancel the fine-tune)
[2023-03-28 15:49:45] Created fine-tune: ft-MPF2xw4JKKURr7gU4EsxRERI

Stream interrupted (client disconnected).
To resume the stream, run:

  openai api fine_tunes.follow -i ft-MPF2xw4JKKURr7gU4EsxRERI



In [22]:
!python3 /Users/chrispriebe/Library/Python/3.8/lib/python/site-packages/openai/_openai_scripts.py api fine_tunes.follow -i ft-MPF2xw4JKKURr7gU4EsxRERI

[2023-03-28 15:49:45] Created fine-tune: ft-MPF2xw4JKKURr7gU4EsxRERI
[2023-03-28 15:52:43] Fine-tune costs $3.49
[2023-03-28 15:52:44] Fine-tune enqueued. Queue number: 0
[2023-03-28 15:52:46] Fine-tune started
[2023-03-28 15:59:37] Completed epoch 1/4
[2023-03-28 16:05:24] Completed epoch 2/4
[2023-03-28 16:11:12] Completed epoch 3/4
[2023-03-28 16:16:56] Completed epoch 4/4
[2023-03-28 16:17:19] Uploaded model: curie:ft-personal-2023-03-28-23-17-19
[2023-03-28 16:17:20] Uploaded result file: file-pFzh535nCK6OpPnjWBM65LxZ
[2023-03-28 16:17:20] Fine-tune succeeded

Job complete! Status: succeeded 🎉
Try out your fine-tuned model:

openai api completions.create -m curie:ft-personal-2023-03-28-23-17-19 -p <YOUR_PROMPT>


In [35]:
import openai
from lib.cipher import substitution_cipher
from nltk.translate.bleu_score import sentence_bleu

openai.api_key = os.getenv("OPENAI_API_KEY")

def translate(text):
    response = openai.Completion.create(
        model="curie:ft-personal-2023-03-28-23-17-19",
        prompt=text + "\n\n###\n\n",
        temperature=0.2,
        max_tokens=100,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        stop=["\n###"]
    )
    translation = response.get('choices',[{}])[0].get('text').strip()
    decoded_translation = substitution_cipher(translation, encode=False).strip()
    bleu_score = sentence_bleu([text.split()], decoded_translation.split())
    
    return (translation, decoded_translation, bleu_score)

In [36]:
translate("For God so loved me that he gave me new life in his kingdom")

('Suv Guw zu pumiw ni lxel xi gemi ni rih posi or xoz corgwun',
 'For God so loved me that he gave me new life in his kingdom',
 1.0)

In [37]:
# Try some unknown words
translate("Bongo bongo I love you, gone to Venus with a hole in my shoe")

('Furgu furgu O pumi jua, guri lu Muriz holx e xupi or nj zxuiz',
 'Bongo bongo I love you, gone to Vones with a hole in my shoes',
 0.6997522298221912)